# Carregando Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


# Checando GPU

In [ ]:
!nvidia-smi

Wed Sep 29 13:14:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Carregando Pacotes

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from pathlib import Path
import shutil

# ResNet50

https://keras.io/api/applications/ 

https://keras.io/api/applications/resnet/#resnet50-function

In [ ]:
resnet50 = ResNet50()
resnet50.summary()

102981632/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [ ]:
resnet50_top_false =  ResNet50(include_top=False, input_shape=(224,224,3))
resnet50_top_false.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

# Paramêtros

In [ ]:
# Tamanho imagem
img_rows, img_cols = 224, 224

input_shape = (img_rows, img_cols, 3)

#Batch size

batch_size = 32

#Path base de dados Drive
nome_base = 'oregon_wildlife'
nome_zip = nome_base + '.zip'
dir_path = Path('/content/drive/MyDrive/dataset/')
zip_path = dir_path/nome_zip

#Arquivos npy
nome_modelo = 'resnet50'
!mkdir {nome_modelo}
npy_X_train = nome_modelo + '/X_train.npy' 
npy_y_train = nome_modelo + '/y_train.npy'  
npy_X_test = nome_modelo + '/X_test.npy'
npy_y_test = nome_modelo + '/y_test.npy'

#Arquivos zip

zip_name = nome_modelo + '.zip'
path_zip_caracteristicas = dir_path/zip_name


#Carregar Base de Dados

In [ ]:
!cp {zip_path} .
!unzip -q {nome_zip} # Utilizar as imagens diretamente do Drive é extremamente lento
!rm {nome_zip}

In [ ]:
path_train = '/content/' + nome_base + '/train'
path_test = '/content/' + nome_base + '/test'

In [ ]:
train_dataset = image_dataset_from_directory(
        path_train,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False) # Manter Shuffle = False, pois não haverá treinamento
test_dataset = image_dataset_from_directory(
        path_test,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False)

Found 3136 files belonging to 14 classes.
Found 784 files belonging to 14 classes.


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE) #Otimização
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Criando o extrator

https://keras.io/guides/transfer_learning/

In [ ]:
cnn = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
inputs = keras.Input(shape=input_shape)
x = preprocess_input(inputs)
x = cnn(x)
output = GlobalAveragePooling2D()(x)
model = Model(inputs, output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


# Extraindo Características

In [ ]:
X_train = model.predict(train_dataset)
X_test = model.predict(test_dataset)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(3136, 2048)
(784, 2048)


In [ ]:
y_train = np.concatenate([y for x, y in train_dataset], axis=0)
y_test = np.concatenate([y for x, y in test_dataset], axis=0)

In [ ]:
print(y_train.shape)
print(y_test.shape)

(3136,)
(784,)


In [ ]:
np.save(npy_X_train, X_train)
np.save(npy_y_train, y_train)
np.save(npy_X_test, X_test)
np.save(npy_y_test, y_test)

# Copiando para o Drive

In [ ]:
!zip -r {zip_name} {nome_modelo}

  adding: resnet50/ (stored 0%)
  adding: resnet50/X_train.npy (deflated 15%)
  adding: resnet50/y_test.npy (deflated 96%)
  adding: resnet50/X_test.npy (deflated 15%)
  adding: resnet50/y_train.npy (deflated 99%)


In [ ]:
shutil.copyfile(zip_name, path_zip_caracteristicas)

PosixPath('/content/drive/MyDrive/dataset/resnet50.zip')